# Research Interest Data Prep

In [1]:
import datetime as dt
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.0.0) is available. Your version (1.9.4) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Sean Deering!



### Get Raw Research Interest Data

In [2]:
researchinterest_raw_id = 'syn7117916'
researchinterest = pd.read_csv(syn.get(researchinterest_raw_id).path)

### Process Research Interest Data

In [3]:
#remove brackets
def remove_brackets(col):
    return( col.str.replace(']', '').str.replace('[',''))
researchinterest = researchinterest.apply(remove_brackets, axis=0)

# rename columns
researchinterest = researchinterest.rename(columns={
    'Contact_method':'contact_method',
    'Research_Experience':'research_experience',
    'Research_Interest1':'two_surveys_perday',
    'Research_Interest2':'blood_sample',
    'Research_Interest3':'taking_medication',
    'Research_Interest4':'family_survey',
    'Research_Interest5':'hospital_stay'})

### Filter based on age

In [4]:
underage_participants = pd.read_csv( syn.get('syn21905452').path, sep="\t")
researchinterest = researchinterest[~researchinterest.participantId.isin(underage_participants.participantId)]

### Remove test accounts

In [5]:
test_accounts = pd.read_excel(syn.get('syn21958537').path)

researchinterest = researchinterest[~researchinterest.participantId.isin(test_accounts.participantId)]

### Create Internal & External Copies of the Data

In [6]:
#download the data 
sharing_info = pd.read_excel(syn.get('syn21557215').path)
healthCodes_with_broadsharing = sharing_info[sharing_info.sharing == 'all_qualified_researchers']

EXTERNAL_RESEARCHINTEREST_DATA = researchinterest[researchinterest.participantId.isin(healthCodes_with_broadsharing['participant id'])]
EXTERNAL_RESEARCHINTEREST_DATA.shape

(2305, 9)

### External - Upload to Synapse

In [8]:
SH_EXTERNAL_PROJECT = 'syn18492837'
table_schema_external = Schema(name='Research Interest',
                               columns=as_table_columns(EXTERNAL_RESEARCHINTEREST_DATA),
                               parent=SH_EXTERNAL_PROJECT)
researchinterest_synTable_external = syn.store(Table(table_schema_external,EXTERNAL_RESEARCHINTEREST_DATA))
researchinterest_synTable_external

### Internal - Upload to Synapse

In [ ]:
SH_INTERNAL_PROJECT = 'syn7066726'
table_schema_internal = Schema(name='Research Interest Internal',
                               columns=as_table_columns(researchinterest), 
                               parent=SH_INTERNAL_PROJECT)
researchinterest_synTable_internal = syn.store(Table(table_schema_internal,researchinterest))

### Set Provenance

In [ ]:
activity=Activity(name= 'Research Interest data curation', 
                  description='Process and convert raw data to table format', 
                  used=researchinterest_raw_id, 
                  executed='https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_Research_Interest.ipynb')
#Internal 
syn.setProvenance(researchinterest_synTable_internal, activity)

#External
syn.setProvenance(researchinterest_synTable_external, activity)